# Data cleaning

In [277]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.stats import iqr

In [278]:
df_p = pd.read_csv("all_stocks_5yr.csv")

In [279]:
df_p = df_p[['Name', 'date', 'close']]

In [280]:
df_p.head()

,Name,date,close
0,AAL,2013-02-08,14.75
1,AAL,2013-02-11,14.46
2,AAL,2013-02-12,14.27
3,AAL,2013-02-13,14.66
4,AAL,2013-02-14,13.99


Convert string dates into time object

In [281]:
type(df_p['date'])

pandas.core.series.Series

In [282]:
from datetime import datetime

df_p['date'] = df_p['date'].map(lambda x: datetime.strptime(x, '%Y-%m-%d'))

Ar there any missing values?

In [283]:
np.sum(df_p.isna())

Name     0
date     0
close    0
dtype: int64

In [236]:
# df_p.loc[df_p.isna()['close'] == True]
# df_p = df_p.loc[df_p.isna()['open'] == False]

No missing value.

Are there any outliers?

In [295]:
from scipy.stats import iqr

stock_names = np.unique(df_p['Name'])
n = len(stock_names)
outliers = pd.DataFrame(0, index=np.arange(3*n), columns=['Name','min', 'max'])

for i in range(n):
    stock = df_p[df_p['Name'] == stock_names[i]]
    outliers.loc[i,'Name'] = stock_names[i]
    outliers.loc[i,'min'] = np.mean(stock['close']) - 2.576*np.std(stock['close'])
    outliers.loc[i,'max'] = np.mean(stock['close']) + 2.576*np.std(stock['close'])

outliers.head()

,Name,min,max
0,A,25.435495,72.968555
1,AAL,10.177595,66.608910
2,AAP,51.902448,212.964478
3,AAPL,30.383618,187.749778
4,ABBV,25.509030,96.219850


In [296]:
df_joined = pd.merge(df_p, outliers, on="Name", how="inner")

In [300]:
df = df_joined[(df_joined['close'] > df_joined['min']) & (df_joined['close'] < df_joined['max'])]

In [302]:
len(df)/len(df_joined)

0.9940795425174463

## (a) Mean Yearly Price

In [304]:
mean_df = df[['close']].groupby([df['Name'], df['date'].dt.year]).mean()

## Output

In [161]:
# mean_df = mean_df.rename(columns={'close': 'mean_close'})

In [307]:
df.to_csv("all_stocks")